# 函数最小值的一维搜索

In [1]:
import numpy as np

class OneDOptAlgo(object):
    
    def __init__(self, f, except_inter=1e-3):
        self.f = f # here f is a one dimension function 
        self.except_inter = except_inter

### 1. 黄金分割法

所谓黄金分割法也即为将区间按照黄金分割的比例压缩.

In [2]:
def golden_division(self, inter):
    """
    f is the given function, iter is the interval that variable belongs to
    and err is the error you can tolerate
    """
    rho = (3 - np.sqrt(5))/2
    old_inter = abs(inter[0]- inter[1])
    while old_inter > self.except_inter:
        interval = inter[1] - inter[0]
        new_inter_0 = inter[0] + rho*interval
        new_inter_1 = inter[0] + (1 - rho)*interval
        if self.f(new_inter_0) < self.f(new_inter_1):
            inter = [inter[0], new_inter_1]
        else:
            inter = [new_inter_0, inter[1]]
        old_inter = abs(inter[0] - inter[1])
    return inter

In [3]:
OneDOptAlgo.golden_division = golden_division

In [4]:
# 测试
f = lambda x :x**4 - 14*x**3 +60*x**2 - 70*x
inter = [0, 2]

 
ODAG = OneDOptAlgo(f)
except_interval = ODAG.golden_division(inter)
print('The initiate interval is: \n', inter,
      '\n and the width of the interval is:\n', except_interval)

The initiate interval is: 
 [0, 2] 
 and the width of the interval is:
 [0.780193260011777, 0.7810994677193467]


### 2. 斐波那契数列分割法

斐波那契数列分割法也即为将区间按照斐波那契数列前后数值的比值的比例进行压缩, 可以证明在保证压缩区间对称性的前提下, 斐波那契数列分割法是压缩区间比例最大的.

In [5]:
def fib_division(self, inter, epsilon = 0.05):
    # calculate the Fib serial
    width = inter[1] - inter[0]
    iteration = np.ceil(np.log(width*np.sqrt(5)/self.except_inter)/np.log(1.618)-2)
    count = 0
    a, b = 1, 2
    rho = [a/b]
    for i in range(int(iteration)-1):
        a, b = b, a+b
        rho.append(a/b)
    rho = rho[::-1]

    while count < iteration:
        interval = inter[1] - inter[0]
        if count == iteration - 1: 
            new_inter_0 = inter[0] + (1 - rho[count]-epsilon)*interval
            new_inter_1 = inter[0] + rho[count]*interval
        else:
            new_inter_0 = inter[0] + (1 - rho[count])*interval
            new_inter_1 = inter[0] + rho[count]*interval

        if self.f(new_inter_0) < self.f(new_inter_1):
            inter = [inter[0], new_inter_1]
        else:
            inter = [new_inter_0, inter[1]]
        count += 1
    return inter

In [6]:
OneDOptAlgo.fib_division = fib_division

In [7]:
# 测试
ODAG = OneDOptAlgo(f)
except_interval = ODAG.fib_division(inter)
print('The initiate interval is: \n', inter,
      '\n and the width of the interval is:\n', except_interval)

The initiate interval is: 
 [0, 2] 
 and the width of the interval is:
 [0.780185758513932, 0.7809597523219816]


### 3.二分法

利用函数及其一阶导数, (实际上我们也可以直接比较区间端点的函数值), 来以压缩区间

In [8]:
def bisection(self, inter, diff_f):
    """
    here we use the differentiate of the fucntion f to calculate the argument
    of the minimum value of f
    """
    except_inter = self.except_inter
    old_width = inter[1] - inter[0]
    interation = np.ceil(np.log(old_width/except_inter)/np.log(2))
    for ii in range(int(interation)):
        mid_inter = (inter[0]+inter[1])/2
        if diff_f(mid_inter) > 0:
            inter = [inter[0], mid_inter]
        else:
            inter = [mid_inter, inter[1]]
    return inter

In [9]:
OneDOptAlgo.bisection = bisection

In [10]:
# 测试
ODAG = OneDOptAlgo(f)
except_interval = ODAG.bisection(inter, lambda x:4*x**3 - 14*3*x**2+120*x-70)
print('The initiate interval is: \n', inter,
      '\n and the width of the interval is:\n', except_interval)

The initiate interval is: 
 [0, 2] 
 and the width of the interval is:
 [0.7802734375, 0.78125]


### 4. 牛顿法

实际上是用求导数等于零的点来计算极小值, 当然需要保证二阶导数大于零.

In [11]:
def newton_way(self, init_point, diff1_f, diff2_f):
    except_inter = self.except_inter
    new_point = init_point - diff1_f(init_point)/diff2_f(init_point)
    while abs(new_point - init_point) > except_inter:
        init_point = new_point
        new_point = init_point - diff1_f(init_point)/diff2_f(init_point)
    return new_point

In [12]:
OneDOptAlgo.newton_way = newton_way

In [13]:
# 测试
f = lambda x: x**2/2 - np.sin(x)
diff1 = lambda x:x - np.cos(x)
diff2 = lambda x:1+np.sin(x)
point = OneDOptAlgo(f).newton_way(0.5, diff1, diff2)
print('The initiate point is: \n', 0.5,
      '\n and the best point is:\n', point)

The initiate point is: 
 0.5 
 and the best point is:
 0.7390851339208068


### 5.割线法

以割线近似代表切线, 替代牛顿法, 但是需要两个起始点, 但是不需要二阶导数. 

In [14]:
def secant_way(self, init_point, diff_f):
    x0 =  init_point
    if diff_f(x0) > 0:
        x1 = x0 - 0.1
    else:
        x1 = x0 + 0.1
    x2 = x1 - (x1 - x0)/(diff_f(x1) - diff_f(x0))*diff_f(x1)
    while abs(x2 - x1) > self.except_inter:
        x0, x1 = x1, x2
        x2 = x1 - (x1 - x0)/(diff_f(x1) - diff_f(x0))*diff_f(x1)
    return x2

In [15]:
OneDOptAlgo.secant_way = secant_way

In [16]:
# 测试
point = OneDOptAlgo(f).secant_way(0.5, diff1)
print('The initiate point is: \n', 0.5,
      '\n and the best point is:\n', point)

The initiate point is: 
 0.5 
 and the best point is:
 0.7390845583128391


### comment
### 6.逆抛物线插值法 
更一般地, 注意到牛顿法是需要一个起始点, 及其一阶二阶导数来实现对函数的进行二阶近似, 割线法是通过两个点及其中一点的一阶导数来实现的二阶近似, 我们也可以通过三个点来插值一条抛物线, 从而实现对原函数的二阶近似. 该方法也被称为逆抛物线插值法. 

\begin{equation}
\mathbf{xnew} = \frac{(x_{1}^{2} - x_{2}^{2})f(x_{3}) + (x_{2}^{2} - x_{3}^{2})f(x_{1}) + (x_{3}^{2} - x_{1}^{2})f(x_{2})}{2f(x_{3})(x_{1}-x_{2}) + 2f(x_{1})(x_{2}-x_{3}) + 2f(x_{2})(x_{3}-x_{1})}
\end{equation}

并以此类推, 我们还可以推导出更高阶的公式

In [17]:
def reverse_parabola(self, init_point):
    f = self.f
    x3 = init_point
    x1, x2 = init_point + 0.1, init_point - 0.1
    err = self.except_inter*2
    while err > self.except_inter:
        sig12, sig23, sig31 = x1**2 - x2**2, x2**2 - x3**2, x3**2 - x1**2
        delt12, delt23, delt31 = x1-x2, x2-x3, x3-x1
        xnew = (sig12*f(x3)+sig23*f(x1)+sig31*f(x2))/(2*(delt12*f(x3)+delt23*f(x1)+delt31*f(x2)))
        x3, x2,x1 = xnew, x3, x2
        err = abs(x3-x2)
    return x3

In [18]:
OneDOptAlgo.reverse_parabola = reverse_parabola

In [19]:
point = OneDOptAlgo(f).reverse_parabola(0.5)
print('The initiate point is: \n', 0.5,
      '\n and the best point is:\n', point)

The initiate point is: 
 0.5 
 and the best point is:
 0.7390917542261743
